In [1]:
import numpy as np
import pandas as pd
from scipy.io.wavfile import read, write
import os, glob, shutil
from shutil import copyfile
import pinyin
import librosa

### parameters

In [5]:
path = '/datasets/AVOCADO/raw_small_data'
out_path = '/datasets/AVOCADO/small_data/guoguo'
out_wav_path = os.path.join(out_path, 'wavs')
speaker = '果果'

### clean output directory

In [3]:
shutil.rmtree(out_path, ignore_errors = True)
shutil.rmtree(out_wav_path, ignore_errors = True)
os.mkdir(out_path)
os.mkdir(out_wav_path)

### rename and subsample wavs 

In [6]:
data = []
index = 0
prefix = 'avocado-' + pinyin.get(speaker,format="strip") + '-'
for excel in glob.glob(os.path.join(path, '**','*.xls'),recursive=True):
    df = pd.read_excel(excel).dropna();
    df = df[df['people'] == speaker]
    dir_name = os.path.dirname(excel)
    for idx, row in df.iterrows():
        original_wav_name = os.path.join(dir_name, 'clips', str(row[0]) + ' ' + row[1].strip() + '.wav')
        wav_name = os.path.join(out_wav_path, prefix + str(index) + '.wav')     
        rate, wav = read(original_wav_name) 
        subsampled = librosa.core.resample(wav.astype(float), rate, 16000).astype(np.int16)
        write(wav_name,16000, subsampled)
        data.append([wav_name, pinyin.get(row[1],format="numerical", delimiter=" ")])
        index +=1

In [7]:
df = pd.DataFrame(data)
df.head()

,0,1
0,/datasets/AVOCADO/small_data/guoguo/wavs/avoca...,xi1 xi1 … …
1,/datasets/AVOCADO/small_data/guoguo/wavs/avoca...,zhe4 mei2 shi2 yao1 li4 hai4 de5 la5 ！
2,/datasets/AVOCADO/small_data/guoguo/wavs/avoca...,xiao3 yi4 si1 xiao3 yi4 si1 ！
3,/datasets/AVOCADO/small_data/guoguo/wavs/avoca...,jin1 tian1 ken3 ding4 shi4 lao3 tian1 ye2 si4 ...
4,/datasets/AVOCADO/small_data/guoguo/wavs/avoca...,kao3 shi4 de5 shi2 hou4 ，


### generate train and eval txts

In [8]:
# r = np.random.rand(df.shape[0])
# train = df[r < 0.8]
# evalu = df[r > 0.8]

In [9]:
# for small data experiment
df.to_csv(os.path.join(out_path, "train.txt"), sep='|',index=False, header=None)
#df.to_csv(os.path.join(out_path, "eval.txt"), sep='|',index=False, header=None)